## HST cutouts

In [163]:
import pandas as pd
from astropy.io import fits
import matplotlib.pyplot as plt
# from astropy.table import Table
from ccdproc import CCDData, wcs_project
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
# from scipy.interpolate import griddata
from astropy.visualization import AsinhStretch, PercentileInterval, LogStretch
from astropy.nddata import InverseVariance, StdDevUncertainty
from astropy.stats import sigma_clipped_stats, SigmaClip
from tqdm.notebook import tqdm
from astroscrappy import detect_cosmics
from scipy.interpolate import RegularGridInterpolator
from skimage.transform import resize, rescale
from photutils.background import Background2D, MedianBackground
from astropy.convolution import interpolate_replace_nans, Gaussian2DKernel, Box2DKernel, convolve
from photutils.segmentation import detect_sources
# path = "../data"
# %matplotlib inline

# # For plotting
stretch = LogStretch(10000)
norm    = PercentileInterval(1)

# namecol = "name"
# filt = "i"
example = False
ex = 'J000318+004844'#'J000318+004844'#'J143451+033843'

In [164]:
plt.rcParams['image.interpolation'] = 'None'

In [165]:
data = pd.read_csv('data/catalogs/hst_control_fileinfo.csv')

# Here select a subset to make cutouts for
sample = data
# sample.head()

Make a cutout, align with North

In [172]:
def make_cutout(path, ra, dec, pxscale=None, size=40, align_north=False, rotate_north=False, crreject=False):

    
    file = fits.open(path)
    ccd  = CCDData.read(path, unit=u.electron/u.second)
    wht = file["WHT"].data
    mask = wht == 0
    wcs = ccd.wcs
    
    # If pixel scale is not known, figure it out from the WCS
    if not pxscale:
        pxscale = np.sqrt(np.abs(np.linalg.det(ccd.wcs.wcs.cd))) # Pixel scale in degrees/px
        pxscale *= 3600 # convert to arcsec/px

    # Find a mask of actually bad pixels but not detector area boundary
    detector_area = detect_sources(mask, 0.1, 1e3).data
    mask_nondetector = (mask>0) & (detector_area==0)
    
    # Get values for mean sky and read noise... not always in the same headers, sometimes absent
    if 'PCTERNOI' in ccd.header:
        readnoise = ccd.header['PCTERNOI']
    elif 'PCTERNOI' in file[1].header:
        readnoise = file[1].header['PCTERNOI']
    else:
        readnoise = 5
        
    if 'MDRIZSKY' in ccd.header:
        meansky = ccd.header['MDRIZSKY']
    elif 'MDRIZSKY' in file[1].header:
        meansky = file[1].header['MDRIZSKY']
    else:
        # Calculate from image, which also includes readnoise
        meansky = sigma_clipped_stats(ccd.data)[2]**2 * ccd.header['exptime'] * ccd.header['ccdgain']
        readnoise = 0

    if 'CCDGAIN' in ccd.header: 
        gain = ccd.header['ccdgain']
    else:
        gain = ccd.header['atodgain']
        
    # Calculate the error array from the data instead of weight file
    var = ccd.data + meansky + readnoise**2
    unc = np.sqrt(var)/(gain*ccd.header['exptime'])

    # Reject cosmic rays if needed
    if crreject:
        #### TODO
        #### ADD IN A VARIANCE ARRAY
        #### CHECK IF WHT IS INVERSE VARIANCE LIKE I SAY HERE???
        cr_mask, ccd.data = detect_cosmics(ccd.data*ccd.header['exptime'], gain=gain,
                                   objlim=3, cleantype='meanmask', sigfrac=0.1, niter=6)#, invar=1/unc)
        ccd.data = ccd.data / ccd.header['exptime']
        # mask = mask + cr_mask
        # mask[mask >= 1]   = 1
        # unc[cr_mask > 0] = 0
    
    # Figure out the galaxy center
    cent = skycoord_to_pixel(SkyCoord(ra*u.deg, dec*u.deg), ccd.wcs)
    cent_x, cent_y = int(cent[0]+0.5), int(cent[1]+0.5)

    # Default cutout radius is 40 arcsec
    size = round(size/pxscale)
        
    # Now create the cutout
    xmin = max(0, cent_x-size); xmax = min(cent_x+size, ccd.data.shape[1])
    ymin = max(0, cent_y-size); ymax = min(cent_y+size, ccd.data.shape[0])


    # Make a cutout (this might not be a square or centered on the object due to possible image borders)
    slices = (slice(ymin, ymax), slice(xmin, xmax))
    data  = ccd.data[slices]
    unc = unc[slices]
    mask = mask[slices]
    
    # Interpolate over bad pixels (not CR-rejected)
    mask_nondetector = mask_nondetector[slices]
    data[mask_nondetector] = np.nan
    data = interpolate_replace_nans(data, Gaussian2DKernel(3))

    
    # Pad the array to a desired size
    padx = (max(0,size-cent_x), max(cent_x+size-ccd.data.shape[1], 0))
    pady = (max(0,size-cent_y), max(cent_y+size-ccd.data.shape[0], 0))
    data = np.pad(data, (pady, padx))
    unc = np.pad(unc, (pady, padx))
    mask = np.pad(mask, (pady, padx), constant_values=1)
    slices = (slice(cent_y-size, cent_y+size), slice(cent_x-size, cent_x+size))
    
    # Estimate sky
    sky = sigma_clipped_stats(data)
    data -= sky[1]
    
    # Create the cutout CCD
    ccd_small = ccd.copy()
    ccd_small.data = data
    ccd_small.uncertainty = StdDevUncertainty(unc)
    ccd_small.mask = mask
    
    
    ccd_small.wcs   = wcs.slice(slices)
    ccd_small.header["x_og"] = cent_x
    ccd_small.header["y_og"] = cent_y
    
    imcent = skycoord_to_pixel(SkyCoord(ra*u.deg, dec*u.deg), ccd_small.wcs)
    ccd_small.wcs.wcs.crpix = imcent
    ccd_small.wcs.wcs.crval = np.array([ra, dec])
    
    ccd_small.header["x_og"] = cent_x
    ccd_small.header["y_og"] = cent_y
    ccd_small.header["zp"] = -2.5*np.log10(file['SCI'].header["PHOTFLAM"]) + file['SCI'].header["PHOTZPT"]
    ccd_small.header['pxscale'] = pxscale
    
    # Rotate the image to align it with North (optional)
    if align_north:
        wcs = ccd_small.wcs.deepcopy()
        wcs.wcs.cd = np.array([[0,pxscale/3600],[pxscale/3600,0]])
        ccd_small.data = ccd_small.data.T[::-1]
        ccd_small.uncertainty = StdDevUncertainty(unc.T[::-1])
        ccd_small.mask = mask.T[::-1]
    # Rotate the image to align it with North when the rotation is a simple flip
    elif rotate_north:
        wcs = ccd_small.wcs.deepcopy()
        wcs.wcs.cd = np.array([[0,pxscale/3600],[pxscale/3600,0]])
        
        ccd_rot = wcs_project(ccd_small, wcs)
        
        tmp = ccd_small.copy()
        tmp.data = unc
        ccd_rot.uncertainty = StdDevUncertainty(wcs_project(tmp, wcs).data)

        tmp.data = 1-mask
        mask = wcs_project(tmp, wcs, order='nearest-neighbor').data
        mask = (mask < 0.5) | np.isnan(mask)
        ccd_rot.mask = mask
        
        ccd_small = ccd_rot

    return ccd_small

Example:

In [167]:
if example:
    
    info = sample[sample['name'] == ex].iloc[0]
    file = fits.open(f'data/hst/raw/drc_new/{info.filename}_drc.fits')
    ccd = make_cutout(f'data/hst/raw/drc_new/{info.filename}_drc.fits', info.ra, info.dec, align_north=False, crreject=True)
    plt.imshow(-2.5*np.log10(np.abs(ccd.data/ccd.header['pxscale']**2))+ccd.header['zp'], cmap='gray_r', vmin=20, vmax=30)

In [173]:
for idx, row in tqdm(sample.reset_index().iterrows(), total=len(sample)):
    if idx<36: continue
    # print(row['name'])
    ccd = make_cutout(f'data/hst/raw/{row.filename}_drz.fits', row.ra, row.dec, align_north=False, crreject=False)
    hdu = ccd.to_hdu()
    hdu.writeto(f"data/hst/F814W/{row['name']}.fits", overwrite=True)
    
    
    # try:
    #     fig = plt.figure(figsize=(10,10))
    #     plt.imshow(stretch(norm(ccd)))
    #     plt.savefig(f"data/hst/png/{row['name']}.png", bbox_inches="tight")
    #     plt.close(fig)
    # except:
    #     print(f"======================== \n ERROR IN {row['name']}")

  0%|          | 0/52 [00:00<?, ?it/s]

INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit COUNTS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


...

INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit COUNTS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


INFO: first HDU with data is extension 1. [astropy.nddata.ccddata]
INFO: using the unit electron / s passed to the FITS reader instead of the unit ELECTRONS/S in the FITS file. [astropy.nddata.ccddata]


In [169]:
tmp = fits.open(f'data/hst/raw/{row.filename}_drz.fits')


In [171]:
tmp[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
DATE    = '2009-11-15'         / date this file was written (yyyy-mm-dd)        
FILETYPE= 'SCI      '          / type of data found in data file                
                                                                                
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'WFPC2 '             / identifier for instrument used to acquire data 
EQUINOX =               2000.0 / equinox of celestial coord. system             
                                                                                
              / WFPC-II DATA DESCRIPTOR KEYWORDS                                
                            

## PSF

For each image, get the corresponding PSF centered on the object

In [126]:
psf_file = fits.open('data/hst/raw/psf/PSFSTD_WFC3UV_F814W.fits', ignore_missing_end=True)
psf_arr_flat = psf_file[0].data
psf_file.close()

???????????????????????????????????????????????????????????????????????????????? [astropy.io.fits.card]


Convert the PSF from 101x101x56 to a 101x101x4x7 array.

In [127]:
psf_arr = np.zeros((8,7,101,101))

for nrow in range(8):
    for ncol in range(7):
        psf_arr[nrow, ncol, :, :] = psf_arr_flat[nrow*7 + ncol, :, :]
psf_arr[psf_arr < 0] = 0

# x, y values at the center of each PSF cell
xmax = 4125
ymax = 4385
xs = np.linspace(0, xmax, 8)
ys = np.linspace(0, ymax, 9)
xs = xs[:-1] + np.diff(xs)[0]/2
ys = ys[:-1] + np.diff(ys)[0]/2

psfx = np.arange(101)
psfy = np.arange(101)
psfx = psfx + np.diff(psfx)[0]/2
psfy = psfy + np.diff(psfy)[0]/2

PSFX, PSFY = np.meshgrid(psfx, psfy)

For each galaxy, convert the ra and dec into the pixel value... ignore the rotation for this approximate test, will lead to a slight offset in the PSF.

Given an (x,y) coordinate, for each point in the 101x101 PSF array, interpolate the 7x4 PSF grid to find the PSF at that particular point.

In [128]:
def interpolate_psf(x, y): 
    
    interp = RegularGridInterpolator((ys, xs, psfy, psfx), psf_arr)
    
    pts_psfx = PSFX.flatten()
    pts_psfy = PSFY.flatten()
    
    try:
        pts_x = x*np.ones_like(pts_psfx)
        pts_y = y*np.ones_like(pts_psfy)
        pts = np.stack((pts_y, pts_x, pts_psfy, pts_psfx)).T
        val = interp(pts)
    except:
        pts_x = 2000*np.ones_like(pts_psfx)
        pts_y = 2000*np.ones_like(pts_psfy)
        pts = np.stack((pts_y, pts_x, pts_psfy, pts_psfx)).T
        val = interp(pts)

    val = val.reshape((101,101))
    return val

Now get the PSF for each galaxy

In [129]:
def get_psf(row):
    
    # Get the center of the galaxy's XY position on the original image
    galaxy_f = fits.open(f'data/hst/F814W/{row["name"]}.fits')
    header = galaxy_f[0].header
    x, y = header['X_OG'], header['Y_OG']
    
    # PSF oversampled by a factor of 4
    psf = interpolate_psf(x, y)
    
    # Reshape back to the original pixel scale
    psf_small = resize(psf, (25,25), order=3)
    
    # Write both as an HDU
    header = fits.Header()
    header['EXTNAME'] = 'PSF'
    psf_hdu = fits.ImageHDU(data=psf_small, header=header)
    header['EXTNAME'] = 'PSF_X4'
    psf_large_hdu = fits.ImageHDU(data=psf, header=header)
    
    # Write to file
    galaxy_f.append(psf_hdu)
    galaxy_f.append(psf_large_hdu)
    galaxy_f.writeto(f'data/hst/F814W/{row["name"]}.fits', overwrite=True)
    

In [130]:
for idx, row in data.iterrows():
    get_psf(row)

## Control galaxies

Control galaxies are mostly ACS, some are WFPC3, so we use TinyTim PSFs instead. Some are still WFC3 so in the WFC3 case we do as for the spogs. All three WFPC3 galaxies are taken with planetary camera (chip 1).

* Make a default parameter file with ACS/WFPC3 settings (`acs1.param`, `acs2.param`, `wfpc.param`)
* Change object-specific parameters
    * Line 2: output file name should be galaxy name
    * Line 14: should be `X Y` for the galaxy position

In [174]:
sample = data#[data.name.isin(names)]

In [176]:
! TINYTIM=/Users/liza/software/tinytim-7.5
! export TINYTIM

for idx, row in tqdm(sample.iterrows(), total=len(sample)):
    
    if 'wfc3' in row.filename:
        get_psf(row)
    # Use TinyTim to generate the PSF
    else:
        file = fits.open(f'data/hst/F814W/{row["name"]}.fits')
        header = file[0].header
        x, y = header['X_OG'], header['Y_OG']
        
        # Decide which default parameters file to use
        if 'acs' in row.filename:
            param_file = 'acs1' if 'WFC1' in header["PHOTMODE"] else 'acs2'
        else:
            param_file = 'wfpc'
        
        # Crete a new parameter file with the current x, y and filename
        with open(f'data/hst/raw/psf/{param_file}.par') as f:
            lines       = f.readlines()
        lines[1]    = f'data/hst/raw/psf/{row["name"]}\n'
        lines[13]   = f'{x} {y}\n'
        with open(f'data/hst/raw/psf/{row["name"]}.par', "w+") as f:
            f.writelines(lines)
        
        # # Run TinyTim
        # command1 = f"/Users/liza/software/tinytim-7.5/tiny2 data/hst/raw/psf/{row['name']}.par" 
        # ! {command1}
        # command2 = f"/Users/liza/software/tinytim-7.5/tiny3 data/hst/raw/psf/{row['name']}.par" 
        # ! {command2}
        
        # Write the PSF to the fits file
        psf_file = fits.open(f"data/hst/raw/psf/{row['name']}00.fits")
        psf_file[0].header['EXTNAME'] = 'PSF'
        file.append(psf_file[0])
        file.writeto(f'data/hst/F814W/{row["name"]}.fits', overwrite=True)

        

  0%|          | 0/52 [00:00<?, ?it/s]

## Fix the file

We want all data to have a 4-extension FITS file, each HDU containing image -> err -> mask -> psf respectively. We also want each HDU to have an `EXTNAME` equal to `IMAGE`, `ERR`, `MASK`, or `PSF`.

In [177]:
for idx, row in tqdm(data.iterrows()):
    
    galaxy_f = fits.open(f'data/hst/F814W/{row["name"]}.fits')
        
    # Fix order and add extanmes
    try:
        hdus = [galaxy_f['SCI'], galaxy_f['UNCERT'], galaxy_f['MASK'], galaxy_f['PSF']]
        names = ['IMAGE', 'ERR', 'MASK', 'PSF']
        for hdu, name in zip(hdus, names):
            hdu.header['EXTNAME'] = name

    #     # Save file
        hdul = fits.HDUList(hdus)
        hdul.writeto(f'data/hst/F814W/{row["name"]}.fits', overwrite=True)
        print(row['name'])
    except:
        continue
        
    
    galaxy_f.close()    

0it [00:00, ?it/s]

J002232+002127
J082517+210940
J085254+334754
J091516+421153
J094109+344202
J095752+022908
J095757+020656
J095800+022811
J095853+022603
J095859+021500
J095902+021127
J095908+024213
J095926+020856
J095944+024914
J095947+024913
J100008+014212
J100047+023305
J100130+021705
J100130+025141
J100142+022520
J100154+022233
J100218+015124
J102000+081334
J104506+213136
J105922+243023
J112228+341431
J122705+333644
J123704+260525
J124810-014449
J125950+275529
J130041+275342
J135606+051031
J141910+525151
J141927+525517
J142650+333507
J142830+332954
J142912+335713
J143403+333519
J143451+033843
J153711+412555
J160210+155639
J160210+155928
J161017+531456
J162904+470853
J171532+572251
J171813+593158
J172103+593313
J172338+591647
J211823+001730
J232621-000237
J234116+000021
J234120+000043
